# Importing Packages

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from google.colab import drive

# Mounting Drive

In [2]:
drive.mount('/content/drive/')

Mounted at /content/drive/


# Importing File

In [3]:
location = '/content/drive/My Drive/Gym'
file_path = location+"/strong.csv"
gym_data = pd.read_csv(file_path)

# Functions

In [4]:
def convert_duration_to_minutes(duration_str):
    total_minutes = 0
    parts = duration_str.split()
    for part in parts:
        if 'h' in part:
            total_minutes += int(part.strip('h')) * 60
        elif 'm' in part:
            total_minutes += int(part.strip('m'))
    return total_minutes

def format_date(input_date):
    # Convert the input date string to a datetime object
    date_obj = datetime.strptime(input_date, "%Y-%m-%d %H:%M:%S")

    # Extract day and month information
    day = date_obj.day
    month = date_obj.strftime("%B")

    # Add 'st', 'nd', 'rd', or 'th' to the day based on its value
    if 10 <= day % 100 <= 20:
        day_suffix = "th"
    else:
        day_suffix = {1: "st", 2: "nd", 3: "rd"}.get(day % 10, "th")

    # Format the date string
    formatted_date = f"{day}{day_suffix} of {month}"

    return formatted_date

def get_first_index(df, col, val):
    return df[df[col]==val].index[0]

def convert_numeric_to_day(num):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
    if 0 <= num <= 6:
        return days[num]
    else:
        return "Invalid Day"

In [5]:
def strong_wrapped(df, year):

    # CONVERT DATE TO DATETIME OBJECT
    df["datetime"] = pd.to_datetime(df["Date"])

    # FILTER BY YEAR
    df = df[df["datetime"].dt.year==year]

    # TOTAL NUM SESSIONS
    num_sessions = len(df["Date"].unique())

    # Only proceed if enough data
    if num_sessions <= 5:
        return "Please specify a year with more than 5 sessions!"

    # Create list of beginning indexes for each workout
    index_list = []
    for unique_date in df["Date"].unique():
        index_list.append(get_first_index(df, "Date", unique_date))
    # Convert all durations into minutes
    duration_df = df.loc[index_list]["Duration"].apply(convert_duration_to_minutes)

    # TOTAL DURATION DATA
    total_duration =  duration_df.sum()
    sandman_equivalents = total_duration / 5.51666666666

    # TOTAL WEIGHT DATA
    total_weight_moved = df["Weight"].sum()
    ronnie_weight_equivalents = np.round((total_weight_moved / 133),2)

    # MOST FREQUENT DAY / HOUR
    hour_mode = df["datetime"].dt.hour.mode()[0]
    favourite_day = convert_numeric_to_day(df["datetime"].dt.weekday.mode()[0])

    # EXERCISE FREQ DATA
    num_exercises = len(df["Exercise Name"].unique())
    val_counts = df["Exercise Name"].value_counts()
    avg_ex_sets = val_counts.mean()

    # EXERCISE TOTALS BY SETS, WEIGHTS, REPS
    exercise_totals = []
    for exercise in df["Exercise Name"].unique():
        total_sets = len(df[df["Exercise Name"]==exercise])
        total_weight = df[df["Exercise Name"]==exercise]["Weight"].sum()
        total_reps = df[df["Exercise Name"]==exercise]["Reps"].sum()
        exercise_totals.append([exercise, total_sets, total_weight, total_reps])

    # PUT INTO DF
    ex_totals_df = pd.DataFrame(exercise_totals, columns=["Exercise", "Total Sets", "Total Weight", "Total Reps"])

    # TOP 3 or N exercises by metric, whichever is smaller
    n = 3
    if len(val_counts)<3:
        n = len(val_counts)

    # TOP BY SETS
    top_n_sets = ex_totals_df.nlargest(3, "Total Sets").reset_index(drop=True)
    top_set_out = ""
    for i in range(n):
        top_set_out += "    - " + top_n_sets["Exercise"].loc[i] + " for " + str(top_n_sets["Total Sets"].loc[i]) +" sets.\n"

    # TOP BY WEIGHT
    top_n_weight = ex_totals_df.nlargest(3, "Total Weight").reset_index(drop=True)
    top_weight_out = ""
    for i in range(n):
        top_weight_out += "    - " + top_n_weight["Exercise"].loc[i] + " with " + str(np.round((top_n_weight["Total Weight"].loc[i]/1000),1)) +" metric tonnes.\n"

    # TOP BY REPS
    top_n_reps = ex_totals_df.nlargest(3, "Total Weight").reset_index(drop=True)
    top_reps_out = ""
    for i in range(n):
        top_reps_out += "    - " + top_n_reps["Exercise"].loc[i] + " for " + str(top_n_reps["Total Reps"].loc[i]) +" reps.\n"

    # BARBELL PR
    barbell_pr = ""
    barbell_df = df[df['Exercise Name'].str.contains('(Barbell)', regex=False)]
    if len(barbell_df) > 0:
        max_idx = barbell_df["Weight"].idxmax()
        pr_df = barbell_df.loc[max_idx]
        pr_name = pr_df["Exercise Name"].replace('(Barbell)', '')
        pr_reps = pr_df["Reps"]
        if pr_reps > 1:
            pr_reps = str(pr_df["Reps"]) + " reps."
        else:
            pr_reps = str(pr_df["Reps"]) + " rep."
        barbell_pr += "on the " + format_date(pr_df["Date"]) + " with " + str(np.round(pr_df["Weight"],2)) +"kg in the " + pr_name + "for " + pr_reps

    # DUMBBELL PR
    dumbbell_pr = ""
    dumbbell_df = df[df['Exercise Name'].str.contains('(Dumbbell)', regex=False)]
    if len(dumbbell_df) > 0:
        max_idx = dumbbell_df["Weight"].idxmax()
        pr_df = dumbbell_df.loc[max_idx]
        pr_name = pr_df["Exercise Name"].replace('(Dumbbell)', '')
        pr_reps = pr_df["Reps"]
        if pr_reps > 1:
            pr_reps = str(pr_df["Reps"]) + " reps."
        else:
            pr_reps = str(pr_df["Reps"]) + " rep."
        dumbbell_pr += "on the " + format_date(pr_df["Date"]) + " with " + str(np.round(pr_df["Weight"],2)) +"kg in the " + pr_name + "for " + pr_reps

    out_str = "Welcome to your "+ str(year)+ " Strong Wrapped!\n"
    out_str += "\nThis year you recorded " + str(num_sessions) + " sessions!"
    out_str += "\nMostly you worked out at " + str(hour_mode) +":00, and your favourite day for a session was " + favourite_day
    out_str += "\nThese sessions add up to a total of " + str(total_duration) + " minutes, which is " +  str(np.round((total_duration/1440),1)) + " days,"
    out_str += " or " + str(np.round(sandman_equivalents,1)) + " loops of 'Enter Sandman'\n"
    out_str += "\nIn total, you've moved " + str(np.round((total_weight_moved/1000),1)) + " metric tonnes, which is equivalent to " + str(ronnie_weight_equivalents) + " contest weight Ronnie Coleman's."
    out_str += "\n\nThis year, you've tried " + str(num_exercises) + " different exercises -"
    if num_exercises > 50:
        out_str += " how adventurous."
    else:
        out_str += " maybe it's time to explore a bit?"
    out_str += "\nYou've done each exercise for an average of " + str(np.round(avg_ex_sets,1)) + " sets."
    out_str += "\n\nYour top " + str(n) + " exercises this year by number of sets have been: \n" + top_set_out
    out_str += "\nYour top " + str(n) + " exercises this year by total weight have been: \n" + top_weight_out
    out_str += "\nYour top " + str(n) + " exercises this year by number of reps have been: \n" + top_reps_out
    out_str += "\nYour heaviest barbell PR was " + barbell_pr
    out_str += "\nYour heaviest dumbbell PR was " + dumbbell_pr
    return out_str

# Use

In [6]:
print(strong_wrapped(gym_data, 2023))

Welcome to your 2023 Strong Wrapped!

This year you recorded 113 sessions!
Mostly you worked out at 13:00, and your favourite day for a session was Saturday
These sessions add up to a total of 9939 minutes, which is 6.9 days, or 1801.6 loops of 'Enter Sandman'

In total, you've moved 103.7 metric tonnes, which is equivalent to 779.8 contest weight Ronnie Coleman's.

This year, you've tried 100 different exercises - how adventurous.
You've done each exercise for an average of 27.6 sets.

Your top 3 exercises this year by number of sets have been: 
    - Bench Press (Barbell) for 245 sets.
    - Lat Pulldown (Machine) for 180 sets.
    - Bent Over One Arm Row (Dumbbell) for 129 sets.

Your top 3 exercises this year by total weight have been: 
    - Bench Press (Barbell) with 13.3 metric tonnes.
    - Lat Pulldown (Machine) with 13.1 metric tonnes.
    - Chest Fly with 10.3 metric tonnes.

Your top 3 exercises this year by number of reps have been: 
    - Bench Press (Barbell) for 1757 re